In [2]:
import bs4
import requests
import numpy as np, pandas as pd
import time
import html5lib
import re

In [2]:
START_SEASON = "2017-2018"

In [3]:
def get_fbref_data(url: str, season: str) -> list|None:

		response = requests.get(url)
		results = []
		
		# Check if the request was successful
		if response.status_code == 200:
			# Parse the HTML content
			soup = bs4.BeautifulSoup(response.text, 'html.parser')
			table_data = None
			comments = soup.find_all(string=lambda text: isinstance(text, bs4.Comment))

			# Look for the specific div in the comments
			for comment in comments:
				comment_soup = bs4.BeautifulSoup(comment, 'html.parser')
				div_standard_stats = comment_soup.find('div', id='div_stats_standard')
				if div_standard_stats:
					table_data = div_standard_stats.find('table')
			
			# Find the div with the class 'all-stats-standard'
			div = table_data
			
			# Find the table within the div
			if div:
				# Find the rows in the table
				name_col = div.find_all('td', {'data-stat': 'player'})
				match_logs_col = div.find_all('td', {'data-stat': 'matches'})
				position_col = div.find_all('td', {'data-stat': 'position'})
				birth_year_col = div.find_all('td', {'data-stat': 'birth_year'})
				nationality_col = div.find_all('td', {'data-stat': 'nationality'})
				team_col = div.find_all('td', {'data-stat': 'team'})

				player_name = ""
				player_href = ""
				id = ""
				match_logs_href = ""

				if len(name_col) < 1:
					print(f'Failed to find the player column for season {season}.')
					return None

				for idx in range(len(name_col)):
					player_name = name_col[idx].find('a').get_text()
					id = name_col[idx].get('data-append-csv')
					position = position_col[idx].get_text()
					match_logs_href = match_logs_col[idx].find('a')['href']
					year_of_birth = birth_year_col[idx].get_text()
					nationality = nationality_col[idx].find('a').find('span').get_text()
					team = team_col[idx].find('a').get_text()

					print({
						'season': season,
						'player_name': player_name,
						'position': position,
						'team': team,
						"nationality": nationality,
						'year_of_birth': year_of_birth,
						'match_logs_href': "https://fbref.com" + match_logs_href,
						'fbref_id': id
					})
					results.append({
						'season': season,
						'player_name': player_name,
						'position': position,
						'team': team,
						"nationality": nationality,
						'year_of_birth': year_of_birth,
						'match_logs_href': "https://fbref.com" + match_logs_href,
						'fbref_id': id
					})
				
				print(f'SUCCESS')
				print(f'Number of players found: {len(name_col)}')
				print("----------------------------------------")
				print("\n")
				return results
			else:
				print(f'Failed to find the div for season {season}.')
				return None

		else:
			print(response.status_code)
			print(response.text)
			print(f'Failed to retrieve the HTML content for season {season}.')
			return None

In [4]:
# # Initialize an empty list to store the results
# all_results = []

# # Iterate through the seasons from 2000-2001 to 2024-2025
# for year in range(2000, 2025):
# 	time.sleep(5)
# 	try:
# 		season = f"{year}-{str(year + 1)}"
# 		url = f'https://fbref.com/en/comps/9/{season}/stats/{season}-Premier-League-Stats'
# 		print(f'Processing season {season} at URL: {url}', end=" ... ")
		
# 		season_results = get_fbref_data(url, season)
# 		# Check if the request was successful
# 		if season_results is None:
# 			print(f'Failed to retrieve the data for season {season}.')
# 			continue
# 		all_results.extend(season_results)

# 	except Exception as e:
# 		print(f'An error occurred while processing season {season}: {e}')

# # Convert the results to a DataFrame
# df = pd.DataFrame(all_results)
# df = df.drop_duplicates(subset=['fbref_id', 'season'], keep='first')
# df = df[['fbref_id', 'player_name', 'position', 'match_logs_href', 'season']]

# # Save the results to a CSV file
# df.to_csv('./fbref_data/player_data.csv', index=False)

Processing season 2000-2001 at URL: https://fbref.com/en/comps/9/2000-2001/stats/2000-2001-Premier-League-Stats ... 

/var/folders/g6/gw4mzsv14sldb35rlls_j_f80000gn/T/ipykernel_14477/3740053764.py:15: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  comment_soup = bs4.BeautifulSoup(comment, 'html.parser')


{'season': '2000-2001', 'player_name': 'Nabil Abidallah', 'position': 'MF', 'match_logs_href': 'https://fbref.com/en/players/e32b9fc3/matchlogs/2000-2001/summary/Nabil-Abidallah-Match-Logs', 'fbref_id': 'e32b9fc3'}
{'season': '2000-2001', 'player_name': 'Clarence Acuña', 'position': 'MF', 'match_logs_href': 'https://fbref.com/en/players/cbf4c181/matchlogs/2000-2001/summary/Clarence-Acuna-Match-Logs', 'fbref_id': 'cbf4c181'}
{'season': '2000-2001', 'player_name': 'Tony Adams', 'position': 'DF', 'match_logs_href': 'https://fbref.com/en/players/26b5e727/matchlogs/2000-2001/summary/Tony-Adams-Match-Logs', 'fbref_id': '26b5e727'}
{'season': '2000-2001', 'player_name': 'Ade Akinbiyi', 'position': 'FW', 'match_logs_href': 'https://fbref.com/en/players/6a723914/matchlogs/2000-2001/summary/Ade-Akinbiyi-Match-Logs', 'fbref_id': '6a723914'}
{'season': '2000-2001', 'player_name': 'Rati Aleksidze', 'position': 'FW', 'match_logs_href': 'https://fbref.com/en/players/c4e7060b/matchlogs/2000-2001/summa

In [11]:
data = pd.read_csv('./fbref_data/player_data.csv', index_col=False)

data = data[data["season"] >= START_SEASON].reset_index(drop=True)

data.head()

,fbref_id,player_name,position,match_logs_href,season
0,d60c03a9,Joe White,MF,https://fbref.com/en/players/d60c03a9/matchlog...,2023-2024
1,dd323728,Neco Williams,DF,https://fbref.com/en/players/dd323728/matchlog...,2023-2024
2,8b9ebd03,Willian,FW,https://fbref.com/en/players/8b9ebd03/matchlog...,2023-2024
3,a3b03921,Joe Willock,MF,https://fbref.com/en/players/a3b03921/matchlog...,2023-2024
4,c596fcb0,Callum Wilson,FW,https://fbref.com/en/players/c596fcb0/matchlog...,2023-2024


In [12]:
def get_match_log_data(name: str, fbref_id: str, url: str, season: str, **kwargs) -> dict:
	""""""
	retries = kwargs.get('retries', 3)
	stat_type = kwargs.get('stat_type', 'summary')
	is_goalkeeper = kwargs.get('goalkeeper', False)

	columns = {
		"summary": ['minutes', 'goals', 'assists', 'pens_made', 'pens_att', 'shots', 'shots_on_target', 'cards_yellow', 'cards_red', 'blocks', 'xg', 'npxg'],
		"passing": ["passes_total_distance", "passes_progressive_distance", "passes_completed_short", "passes_short", "passes_completed_medium", "passes_medium", "passes_completed_long", "passes_long", "xg_assist", "pass_xa", "assisted_shots", "passes_into_final_third", "passes_into_penalty_area", "crosses_into_penalty_area", "progressive_passes"],
		"gca": ["sca", "sca_passes_live", "sca_passes_dead", "sca_take_ons", "sca_shots", "sca_fouled", "sca_defense", "gca", "gca_passes_live", "gca_passes_dead", "gca_take_ons", "gca_shots", "gca_fouled", "gca_defense"],
		"defense": ["tackles", "tackles_won", "tackles_def_3rd", "tackles_mid_3rd", "tackles_att_3rd", "challenge_tackles", "challenges", "blocked_shots", "blocked_passes", "interceptions", "clearances", "errors"],
		"possession": ["touches", "touches_def_pen_area", "touches_def_3rd", "touches_mid_3rd", "touches_att_3rd", "touches_att_pen_area", "touches_live_ball", "take_ons", "take_ons_won", "take_ons_tackled", "carries", "carries_distance", "carries_progressive_distance", "progressive_carries", "carries_into_final_third", "carries_into_penalty_area", "miscontrols", "dispossessed", "passes_received", "progressive_passes_received" ],
		"keeper": ["gk_shots_on_target_against", "gk_goals_against", "gk_saves", "gk_clean_sheets", "gk_psxg", "gk_pens_att", "gk_pens_allowed", "gk_pens_saved", "gk_pens_missed", "gk_passed_completed_launched", "gk_passes_launched", "gk_passes", "gk_passes_throws", "gk_passes_length_avg", "gk_goal_kicks", "gk_goal_kicks_length_avg"],
	}

	results = []

	match_logs_table_id = "matchlogs_all"

	if not is_goalkeeper and stat_type == "keeper":
		# Find out the number of rows required for this player
		url = url.replace("keeper", "summary")

	response = requests.get(url)
	
	if response.status_code == 200:
		soup = bs4.BeautifulSoup(response.text, 'html.parser')

		# Attempt to find the table directly or within comments
		table_data = soup.find('table', id=match_logs_table_id)
		if table_data is None:
			comments = soup.find_all(string=lambda text: isinstance(text, bs4.Comment))
			table_data = next(
				(bs4.BeautifulSoup(comment, 'html.parser').find('table', id=match_logs_table_id) 
					for comment in comments), None
			)

		if table_data:
			rows = table_data.find_all('tr', class_=lambda x: x is None or 'unused_sub' not in x.split())

			for row in rows[2:-1]:
				if not row.find('th', {'data-stat': 'date'}).get_text(strip=True):
					continue

				match_data = {
					'fbref_id': fbref_id,
					'player_name': name,
					'season': season,
					'date': row.find('th', {'data-stat': 'date'}).get_text(strip=True) if row.find('th', {'data-stat': 'date'}) else None,
					'competition': row.find('td', {'data-stat': 'comp'}).get_text(strip=True) if row.find('td', {'data-stat': 'comp'}) else None,
					'gameweek': row.find('td', {'data-stat': 'round'}).get_text(strip=True) if row.find('td', {'data-stat': 'round'}) else None,
					'location': row.find('td', {'data-stat': 'venue'}).get_text(strip=True) if row.find('td', {'data-stat': 'venue'}) else None,
					'result': row.find('td', {'data-stat': 'result'}).get_text(strip=True) if row.find('td', {'data-stat': 'result'}) else None,
					'team': row.find('td', {'data-stat': 'team'}).get_text(strip=True) if row.find('td', {'data-stat': 'team'}) else None,
					'opponent': row.find('td', {'data-stat': 'opponent'}).get_text(strip=True) if row.find('td', {'data-stat': 'opponent'}) else None,
					'position': row.find('td', {'data-stat': 'position'}).get_text(strip=True) if row.find('td', {'data-stat': 'position'}) else None,
					'started': row.find('td', {'data-stat': 'game_started'}).get_text(strip=True) if row.find('td', {'data-stat': 'game_started'}) else None,
				}

				if not is_goalkeeper and stat_type == "keeper":
					# Set all the goalkeeper stats to 0 for non-goalkeepers
					for column in columns[stat_type]:
						match_data[column] = 0

				else:
					for column in columns[stat_type]:
						match_data[column] = row.find('td', {'data-stat': column}).get_text(strip=True) if row.find('td', {'data-stat': column}) else None
						match_data[column] = float(match_data[column]) if match_data[column] else 0
			
				results.append(match_data)

	else:
		print(f'F', end=" ")
		time.sleep(10)
		retries -= 1
		if retries > 0:
			retry_result = get_match_log_data(name, fbref_id, url, season, retries=retries)
			if "missing_data" in retry_result:
				print(f'')
				return retry_result
			if "data" in retry_result:
				return retry_result
			return {"data": retry_result}
		else:
			return {
				"missing_data" : {
				'fbref_id': fbref_id,
				'player_name': name,
				'season': season,
				'url': url,
				'error': f'Failed to retrieve the HTML content. Error code {response.status_code}'
				}
			}
	print('SUCCESS')
	return {"data": results}

In [13]:
# # The granular stat types in the match logs begin in 2017/2018 season [Standard, Passing, Defensive Actions, Possession]  

# match_logs_results = []
# enhanced_match_logs_results = []

# missing_data = []
# missing_enhanced_data = []

# for idx, row in data.iterrows():
# 	fbref_id = row["fbref_id"]
# 	name = row["player_name"]
# 	position = row["position"]
# 	url = row["match_logs_href"]
# 	season = row["season"]
# 	retries = 3

# 	urls = []

# 	if season >= "2017-2018":
# 		summary_url = url
# 		urls.append(summary_url)
# 		passing_url = url.replace("summary", "passing")
# 		urls.append(passing_url)
# 		defensive_actions_url = url.replace("summary", "defense")
# 		urls.append(defensive_actions_url)
# 		possession_url = url.replace("summary", "possession")
# 		urls.append(possession_url)
# 		gca_url = url.replace("summary", "gca")
# 		urls.append(gca_url)
# 		goalkeeping_url = url.replace("summary", "keeper")
# 		urls.append(goalkeeping_url)

# 	if season >= "2017-2018":
# 		enhanced_stats = []
# 		for url in urls:
# 			print(f'({idx}) Processing player {name} logs for season {season} at URL: {url}', end=' ... ')
# 			try:
# 				result = get_match_log_data(name, fbref_id, url, season, retries=retries, stat_type=url.split("/")[-2], goalkeeper=position == "GK")
# 				if "data" in result:
# 					if len(enhanced_stats) == 0:
# 						enhanced_stats = result["data"]
# 					else:
# 						for i, match in enumerate(result["data"]):
# 							enhanced_stats[i].update(match)
# 				elif "missing_data" in result:
# 					missing_enhanced_data.extend(result["missing_data"])
# 			except Exception as e:
# 				print(f'FAILED - {e.__cause__}: {str(e)}')
# 				missing_enhanced_data.append({
# 					'fbref_id': fbref_id,
# 					'player_name': name,
# 					'position': position,
# 					'season': season,
# 					'url': url,
# 					'error': str(e)
# 				})
# 			time.sleep(4)
# 		enhanced_match_logs_results.extend(enhanced_stats)

# 		if idx % 100 == 0:
# 				res = pd.DataFrame(enhanced_match_logs_results)
# 				res.to_csv("./fbref_data/enhanced_player_match_logs.csv", index=False)
# 				print(f'\nSaved data to CSV file\n')

# 	else:
# 		print(f'({idx}) Processing player {name} logs for season {season} at URL: {url}', end=' ... ')

# 		try:
# 			result = get_match_log_data(name, fbref_id, url, season, retries=retries)
# 			if "data" in result:
# 				match_logs_results.extend(result["data"])
# 			elif "missing_data" in result:
# 				missing_data.extend(result["missing_data"])
			
# 			if idx % 100 == 0:
# 				res = pd.DataFrame(match_logs_results)
# 				res.to_csv("./fbref_data/player_match_logs_v2.csv", index=False)
# 				print(f'\nSaved data to CSV file\n')

# 		except Exception as e:
# 			print(f'FAILED - {e.__cause__}: {str(e)}')
# 			missing_data.append({
# 				'fbref_id': fbref_id,
# 				'player_name': name,
# 				'position': position,
# 				'season': season,
# 				'url': url,
# 				'error': str(e)
# 			})
# 		time.sleep(4)

# if len(match_logs_results) > 0:
# 	# Save any remaining data
# 	res = pd.DataFrame(match_logs_results)
# 	res["result"] = res["result"].str.replace("–", "-") # remove ambiguous hyphen character
# 	res.to_csv("./fbref_data/player_match_logs_v2.csv", index=False)

# if len(enhanced_match_logs_results) > 0:
# 	# Save any remaining
# 	res = pd.DataFrame(enhanced_match_logs_results)
# 	res["result"] = res["result"].str.replace("–", "-") # remove ambiguous hyphen character
# 	res.to_csv("./fbref_data/enhanced_player_match_logs.csv", index=False)

(0) Processing player Joe White logs for season 2023-2024 at URL: https://fbref.com/en/players/d60c03a9/matchlogs/2023-2024/summary/Joe-White-Match-Logs ... SUCCESS
(0) Processing player Joe White logs for season 2023-2024 at URL: https://fbref.com/en/players/d60c03a9/matchlogs/2023-2024/passing/Joe-White-Match-Logs ... SUCCESS
(0) Processing player Joe White logs for season 2023-2024 at URL: https://fbref.com/en/players/d60c03a9/matchlogs/2023-2024/defense/Joe-White-Match-Logs ... SUCCESS
(0) Processing player Joe White logs for season 2023-2024 at URL: https://fbref.com/en/players/d60c03a9/matchlogs/2023-2024/possession/Joe-White-Match-Logs ... SUCCESS
(0) Processing player Joe White logs for season 2023-2024 at URL: https://fbref.com/en/players/d60c03a9/matchlogs/2023-2024/gca/Joe-White-Match-Logs ... SUCCESS
(0) Processing player Joe White logs for season 2023-2024 at URL: https://fbref.com/en/players/d60c03a9/matchlogs/2023-2024/keeper/Joe-White-Match-Logs ... SUCCESS

Saved data 

In [14]:
# print(len(missing_enhanced_data))
# print(missing_enhanced_data)

# if len(missing_data) > 0:
# 	pd.DataFrame(missing_enhanced_data).to_csv("./fbref_data/missing_enhanced_match_logs_data.csv", index=False)

0
[]


In [20]:
all_match_logs = pd.read_csv('./app/backend/database/ingestion/data/fbref_data/enhanced_player_match_logs.csv')
all_match_logs.head()

,fbref_id,player_name,season,date,competition,gameweek,location,result,team,opponent,...,gk_pens_allowed,gk_pens_saved,gk_pens_missed,gk_passed_completed_launched,gk_passes_launched,gk_passes,gk_passes_throws,gk_passes_length_avg,gk_goal_kicks,gk_goal_kicks_length_avg
0,5f09991f,Patrick van Aanholt,2017-2018,2017-08-12,Premier League,Matchweek 1,Home,L 0-3,Crystal Palace,Huddersfield,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
1,5f09991f,Patrick van Aanholt,2017-2018,2017-08-19,Premier League,Matchweek 2,Away,L 0-1,Crystal Palace,Liverpool,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
2,5f09991f,Patrick van Aanholt,2017-2018,2017-08-22,EFL Cup,Second round,Home,W 2-1,Crystal Palace,Ipswich Town,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
3,5f09991f,Patrick van Aanholt,2017-2018,2017-08-26,Premier League,Matchweek 3,Home,L 0-2,Crystal Palace,Swansea City,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
4,5f09991f,Patrick van Aanholt,2017-2018,2017-09-10,Premier League,Matchweek 4,Away,L 0-1,Crystal Palace,Burnley,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0


In [21]:
all_match_logs["competition"].unique()

array(['Premier League', 'EFL Cup', 'FA Cup', 'Friendlies (M)',
       'PL2 — Div. 2', 'Serie A', 'WCQ', 'Championship', 'Champions Lg',
       'World Cup', 'PL2 — Div. 1', 'Community Shield', 'La Liga',
       'Copa del Rey', 'Gold Cup', 'DFL-Supercup', 'DFB-Pokal',
       'Bundesliga', 'Europa Lg', 'Russian Premier League', 'Super Cup',
       'FIFA Confederations Cup', 'Trophée des Champions', 'Ligue 1',
       'Coupe de la Ligue', 'Coupe de France', 'Supercopa de España',
       'Ligue 2', 'First League', 'Super League', 'League One',
       '2. Bundesliga', 'Supercoppa', 'MLS', 'First Division A',
       'Süper Lig', 'Eredivisie', 'Coppa Italia', 'Premiership',
       'Série A', 'Sudamericana', 'Primeira Liga', 'Danish Superliga',
       '1. HNL', 'UEFA Nations League', 'UEFA Euro Qualifying',
       'Copa América', 'Africa Cup of Nations qualification',
       'Africa Cup of Nations', 'League Two', 'Asian Cup',
       'National League', 'Super Lg', 'Libertadores', 'Liga Argentina

In [23]:
data = all_match_logs[(all_match_logs["competition"] == "Premier League")]
data

,fbref_id,player_name,season,date,competition,gameweek,location,result,team,opponent,...,gk_pens_allowed,gk_pens_saved,gk_pens_missed,gk_passed_completed_launched,gk_passes_launched,gk_passes,gk_passes_throws,gk_passes_length_avg,gk_goal_kicks,gk_goal_kicks_length_avg
0,5f09991f,Patrick van Aanholt,2017-2018,2017-08-12,Premier League,Matchweek 1,Home,L 0-3,Crystal Palace,Huddersfield,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
1,5f09991f,Patrick van Aanholt,2017-2018,2017-08-19,Premier League,Matchweek 2,Away,L 0-1,Crystal Palace,Liverpool,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
3,5f09991f,Patrick van Aanholt,2017-2018,2017-08-26,Premier League,Matchweek 3,Home,L 0-2,Crystal Palace,Swansea City,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
4,5f09991f,Patrick van Aanholt,2017-2018,2017-09-10,Premier League,Matchweek 4,Away,L 0-1,Crystal Palace,Burnley,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
5,5f09991f,Patrick van Aanholt,2017-2018,2017-09-16,Premier League,Matchweek 5,Home,L 0-1,Crystal Palace,Southampton,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147437,b2626673,Jay Stansfield,2024-2025,2024-08-16,Premier League,Matchweek 1,Away,L 0-1,Fulham,Manchester Utd,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
147438,77d7c96f,Kenny Tete,2024-2025,2024-08-16,Premier League,Matchweek 1,Away,L 0-1,Fulham,Manchester Utd,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
147439,9a28eba4,Adama Traoré,2024-2025,2024-08-16,Premier League,Matchweek 1,Away,L 0-1,Fulham,Manchester Utd,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0
147440,c6dc9ecd,Harry Wilson,2024-2025,2024-08-16,Premier League,Matchweek 1,Away,L 0-1,Fulham,Manchester Utd,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0


In [11]:
data["home_team_id"] = np.where(data["location"] == "Home", data["team"], data["opponent"])
data[['home_team_id', 'team', 'opponent', 'location']].head(20)

/var/folders/g6/gw4mzsv14sldb35rlls_j_f80000gn/T/ipykernel_36442/3082406509.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["home_team_id"] = np.where(data["location"] == "Home", data["team"], data["opponent"])


,home_team_id,team,opponent,location
1606,Rubin Kazan,Rubin Kazan,Krasnodar,Home
1607,Zenit,Rubin Kazan,Zenit,Away
1608,Rubin Kazan,Rubin Kazan,Arsenal Tula,Home
1609,CSKA Moscow,Rubin Kazan,CSKA Moscow,Away
1610,Rubin Kazan,Rubin Kazan,Loko Moscow,Home
1611,SKA-Khabarovsk,Rubin Kazan,SKA-Khabarovsk,Away
1612,Rubin Kazan,Rubin Kazan,Anzhi,Home
1613,Rubin Kazan,Rubin Kazan,Tosno,Home
1616,Spartak Moscow,Rubin Kazan,Spartak Moscow,Away
1617,Rubin Kazan,Rubin Kazan,Ural,Home


In [13]:
data["away_team_id"] = np.where(data["location"] == "Away", data["team"], data["opponent"])
data[["home_team_id", 'away_team_id', 'team', 'opponent', 'location']].head(20)

/var/folders/g6/gw4mzsv14sldb35rlls_j_f80000gn/T/ipykernel_36442/1922937874.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["away_team_id"] = np.where(data["location"] == "Away", data["team"], data["opponent"])


,home_team_id,away_team_id,team,opponent,location
1606,Rubin Kazan,Krasnodar,Rubin Kazan,Krasnodar,Home
1607,Zenit,Rubin Kazan,Rubin Kazan,Zenit,Away
1608,Rubin Kazan,Arsenal Tula,Rubin Kazan,Arsenal Tula,Home
1609,CSKA Moscow,Rubin Kazan,Rubin Kazan,CSKA Moscow,Away
1610,Rubin Kazan,Loko Moscow,Rubin Kazan,Loko Moscow,Home
1611,SKA-Khabarovsk,Rubin Kazan,Rubin Kazan,SKA-Khabarovsk,Away
1612,Rubin Kazan,Anzhi,Rubin Kazan,Anzhi,Home
1613,Rubin Kazan,Tosno,Rubin Kazan,Tosno,Home
1616,Spartak Moscow,Rubin Kazan,Rubin Kazan,Spartak Moscow,Away
1617,Rubin Kazan,Ural,Rubin Kazan,Ural,Home


In [16]:
data["home_team_id"] = np.where(
			(
				(((data["location"] == "Neutral") & (data["result"].str[0] == "W") & (data["result"].str[2].astype(int) > data["result"].str[4].astype(int)))
				| ((data["location"] == "Neutral") & (data["result"].str[0] == "L") & (data["result"].str[4].astype(int) > data["result"].str[2].astype(int)))
				| ((data["location"] == "Neutral") & (data["result"].str[0] == "D"))) & home_team_id.isnull()
			),
			data["team"],
			data["opponent"]
		)
data[['home_team_id', 'team', 'opponent', 'location', 'result']].head(20)

/var/folders/g6/gw4mzsv14sldb35rlls_j_f80000gn/T/ipykernel_36442/2338599439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["home_team_id"] = np.where(


,home_team_id,team,opponent,location,result
1606,Krasnodar,Rubin Kazan,Krasnodar,Home,L 1-2
1607,Zenit,Rubin Kazan,Zenit,Away,L 1-2
1608,Arsenal Tula,Rubin Kazan,Arsenal Tula,Home,W 2-1
1609,CSKA Moscow,Rubin Kazan,CSKA Moscow,Away,W 2-1
1610,Loko Moscow,Rubin Kazan,Loko Moscow,Home,D 1-1
1611,SKA-Khabarovsk,Rubin Kazan,SKA-Khabarovsk,Away,D 1-1
1612,Anzhi,Rubin Kazan,Anzhi,Home,W 6-0
1613,Tosno,Rubin Kazan,Tosno,Home,W 1-0
1616,Spartak Moscow,Rubin Kazan,Spartak Moscow,Away,L 0-1
1617,Ural,Rubin Kazan,Ural,Home,L 0-1


In [28]:
data[data.isnull().any(axis=1)]

/var/folders/g6/gw4mzsv14sldb35rlls_j_f80000gn/T/ipykernel_36442/2860748602.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data[data.isnull().any()]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).